# Part 2.5: SQL Subqueries

Importing sqlite3 to run queries in python:

In [1]:
import pandas as pd
import sqlite3

Connecting to the database and creating a cursor object:

In [2]:
connection = sqlite3.connect('chinook.db')
cursor = connection.cursor()

## Scalar Subqueries in SQL

Subqueries:

In [4]:
query = """

SELECT
    billing_country,
    SUM(total)/(
        SELECT SUM(total)
        FROM invoice
    ) AS country_share
FROM
    invoice
GROUP BY 
    billing_country
ORDER BY
    country_share DESC
LIMIT
    5;

"""

pd.read_sql_query(query,connection)

,billing_country,country_share
0,USA,0.220938
1,Canada,0.113727
2,Brazil,0.090814
3,France,0.082615
4,Germany,0.071053


Filtering with Scalar Subqueries

In [ ]:
query = """

SELECT
    *
FROM
    invoice
WHERE total > (SELECT MAX(total)*0.75
                 FROM invoice
              );
              
"""

pd.read_sql_query(query,connection)

## Multi-row and Multi-column Subqueries in SQL

A scalar subquery:

In [5]:
query = """

SELECT 
    COUNT(*) AS tracks_tally
 -- COUNT the number of rows
FROM 
    track
 -- in the track table
WHERE 
    media_type_id IN 
 -- where the media_type_id is in the following list:
        (SELECT
            media_type_id
         -- Items in the media_type_id column
         FROM 
            media_type
         WHERE
            name LIKE '%MPEG%')
         -- populated based on the name column having MPEG in it.
;

"""

pd.read_sql_query(query,connection)

,tracks_tally
0,3248


Subqueries as Lists:

In [6]:
query = """

SELECT 
    *
-- Select all rows
FROM 
    invoice
-- From the invoice table
WHERE 
    customer_id IN (
-- Where the customer_id is in the following list:
        Select
            customer_id
        FROM 
            customer
        WHERE
            SUBSTRING(first_name,1,1) = 'A');
        -- A list of customer_ids when their first name starts with A.
        
"""

pd.read_sql_query(query,connection)

,invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,123,7,2017-09-23 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,11.88
1,225,7,2018-05-08 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,5.94
2,251,7,2018-07-20 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,6.93
3,337,7,2019-04-03 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,9.90
4,410,7,2019-08-27 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,3.96
5,462,7,2019-12-29 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,8.91
6,472,7,2020-01-31 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,2.97
7,488,7,2020-03-13 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,13.86
8,553,7,2020-08-26 00:00:00,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,4.95
9,135,11,2017-10-25 00:00:00,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,2.97


NOT IN Operator:

In [7]:
query = """

SELECT
    first_name,
    last_name
FROM
    customer
WHERE
    customer_id NOT IN (
        -- A query to make a list of customer_ids whos sum total spent is less than 100 dollars. 
        SELECT
            customer_id
        FROM
            invoice
        GROUP BY
            customer_id
        HAVING
            SUM(total) < 100
    );

"""

pd.read_sql_query(query,connection)

,first_name,last_name
0,Luís,Gonçalves
1,František,Wichterlová
2,Helena,Holý
3,Fernanda,Ramos
4,João,Fernandes
5,Hugh,O'Reilly
6,Manoj,Pareek


Subqueries as Tables:

In [11]:
query = """

SELECT
    AVG(num_sales) as average_sales
FROM
    (SELECT
        billing_city,
        COUNT(*) as num_sales
     FROM 
        invoice
     GROUP BY
        billing_city
     )

"""

pd.read_sql_query(query,connection)

,average_sales
0,11.584906


Multi-Column Subqueries and Joins:

In [13]:
query = """

SELECT
    c.last_name,
    c.first_name,
    i.total_avg
FROM
    customer AS c
JOIN
    (SELECT
        customer_id,
        AVG(total) as total_avg
     FROM
        invoice
     GROUP BY
        customer_id
     ) AS i
ON
    c.customer_id = i.customer_id
LIMIT
    10;
    
"""

pd.read_sql_query(query,connection)

,last_name,first_name,total_avg
0,Gonçalves,Luís,8.376923
1,Köhler,Leonie,7.470000
2,Tremblay,François,11.110000
3,Hansen,Bjørn,8.030000
4,Wichterlová,František,8.030000
5,Holý,Helena,10.725000
6,Gruber,Astrid,7.700000
7,Peeters,Daan,8.627143
8,Nielsen,Kara,3.762000
9,Martins,Eduardo,5.032500


Join Multi-Columns Subqueries:

In [15]:
query = """

SELECT 
    i.billing_country AS country,
    (i.invoice_tally / ct.customer_tally) AS sale_avg_tally
FROM
    (SELECT 
        billing_country, 
        COUNT(*) AS invoice_tally
     FROM 
        invoice
     GROUP BY 
        billing_country) AS i
JOIN
    (SELECT 
        country, 
        COUNT(*) AS customer_tally
     FROM 
        customer
     GROUP BY 
        country) as ct
ON
    i.billing_country = ct.country
ORDER BY
    sale_avg_tally DESC
LIMIT
    10;
    
"""

pd.read_sql_query(query,connection)

,country,sale_avg_tally
0,Czech Republic,15
1,Portugal,14
2,Chile,13
3,Ireland,13
4,Brazil,12
5,Finland,11
6,Spain,11
7,Australia,10
8,Denmark,10
9,France,10


## Nested and Correlated Subqueries in SQL

Filtering with Correlated Subqueries:

In [6]:
query = """

-- SELECT everything from the table
SELECT 
    *
FROM 
    invoice AS i1

-- WHERE the total is greater than the average for that country
WHERE
    total > (SELECT 
                AVG(total)
             FROM 
                 invoice AS i2
             WHERE 
                 i1.billing_country = i2.billing_country)
LIMIT
    5;

"""

pd.read_sql_query(query,connection)

,invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
1,2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.90
2,5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83
3,7,49,2017-01-12 00:00:00,Ordynacka 10,Warsaw,None,Poland,00-358,10.89
4,8,59,2017-01-13 00:00:00,"3,Raj Bhavan Road",Bangalore,None,India,560001,9.90


EXISTS Operators:

In [7]:
query = """

SELECT
    *
FROM
    track t
WHERE NOT EXISTS
    (SELECT 
        * 
     FROM 
        invoice_line i 
     WHERE 
        t.track_id = i.track_id
    );
    
-- Select everything from the track table where that track id doesn't exist in the invoice_line table

"""

pd.read_sql_query(query,connection)

,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
0,99,Your Time Has Come,11,1,4,"Cornell, Commerford, Morello, Wilk",255529,8273592,0.99
1,101,Be Yourself,11,1,4,"Cornell, Commerford, Morello, Wilk",279484,9106160,0.99
2,104,Heaven's Dead,11,1,4,"Cornell, Commerford, Morello, Wilk",276688,9006158,0.99
3,106,Man Or Animal,11,1,4,"Cornell, Commerford, Morello, Wilk",233195,7542942,0.99
4,107,Yesterday To Tomorrow,11,1,4,"Cornell, Commerford, Morello, Wilk",273763,8944205,0.99
...,...,...,...,...,...,...,...,...,...
1692,3498,"Concerto for Violin, Strings and Continuo in G...",342,4,24,Pietro Antonio Locatelli,493573,16454937,0.99
1693,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
1694,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
1695,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


Nested Subqueries:

In [8]:
query = """

-- Find the first and last name of employees 
    -- when the employee has served a customer
        -- who has spent a sum of 100 dollars or more

SELECT 
    first_name, last_name
FROM 
    employee
WHERE 
 -- the employee_id is in a list based on the customer table
    employee_id IN (SELECT
                        support_rep_id
                    FROM 
                        customer
                    WHERE 
                     -- the customer_id is a list based on the invoice table
                        customer_id IN (SELECT 
                                            customer_id
                                        FROM 
                                            invoice
                                        GROUP BY
                                            customer_id
                                        HAVING 
                                         -- spent a total of 100 or more.
                                            SUM(total) > 100.0
                                        )
                    );

"""

pd.read_sql_query(query,connection)

,first_name,last_name
0,Jane,Peacock
1,Margaret,Park
2,Steve,Johnson


In [4]:
query = """

SELECT
  temp.invoice_id,
  SUM(temp.quantity * temp.unit_price) AS total,
  SUM(temp.milliseconds/1000.0/60) AS minute
FROM
  (SELECT 
     il.*,  
     t.genre_id, 
     t.milliseconds
   FROM 
     invoice_line il
   JOIN 
     track t
   ON 
     il.track_id = t.track_id
   WHERE 
     invoice_id IN (SELECT 
                      invoice_id
                    FROM 
                      invoice
                    WHERE
                      billing_country = 'USA')) AS temp
WHERE
  temp.genre_id IN (SELECT
                      genre_id
                    FROM
                      genre
                    WHERE 
                      name LIKE '%Metal%')
GROUP BY
  temp.invoice_id;             

"""

pd.read_sql_query(query,connection)

,invoice_id,total,minute
0,4,1.98,8.788883
1,9,1.98,7.653433
2,17,1.98,9.959167
3,18,0.99,4.379850
4,42,0.99,3.415500
...,...,...,...
61,591,2.97,17.112350
62,595,0.99,4.803467
63,598,1.98,6.268500
64,602,0.99,2.494250


## Common Table Expression in SQL

Using a Single Common Table Expression:

In [6]:
query = """

-- a table containing customer ids and their average spent
WITH customer_avg_table AS (
SELECT customer_id, AVG(total) AS total_avg
FROM invoice
GROUP BY customer_id)

SELECT c.last_name, c.first_name, cat.total_avg
  FROM customer AS c
  JOIN (customer_avg_table) AS cat
    ON c.customer_id = cat.customer_id
LIMIT
10;

"""

pd.read_sql_query(query,connection)

,last_name,first_name,total_avg
0,Gonçalves,Luís,8.376923
1,Köhler,Leonie,7.470000
2,Tremblay,François,11.110000
3,Hansen,Bjørn,8.030000
4,Wichterlová,František,8.030000
5,Holý,Helena,10.725000
6,Gruber,Astrid,7.700000
7,Peeters,Daan,8.627143
8,Nielsen,Kara,3.762000
9,Martins,Eduardo,5.032500


Multiple CTEs:

In [8]:
query = """

WITH
country_invoice_total_table AS (
    SELECT billing_country, SUM(total) AS invoice_total
    FROM invoice
    GROUP BY billing_country),
country_total_table AS (
    SELECT country, COUNT(*) AS customer_tally
    FROM customer
    GROUP BY country)
         
SELECT ct.country, 
        ROUND(i.invoice_total / ct.customer_tally, 2) AS sale_avg
  FROM (country_invoice_total_table) AS i
  JOIN (country_total_table) AS ct
    ON i.billing_country = ct.country
 ORDER BY sale_avg DESC
 LIMIT 5;

"""

pd.read_sql_query(query,connection)

,country,sale_avg
0,Czech Republic,136.62
1,Ireland,114.84
2,Spain,98.01
3,Chile,97.02
4,Portugal,92.57


CTE Constraints:

In [9]:
query = """

WITH 
usa_invoice_table AS (
    SELECT invoice_id 
      FROM invoice 
     WHERE billing_country = 'USA' 
), 
invoice_line_track_table AS ( 
    SELECT il.*, 
           t.milliseconds, 
           t.genre_id
      FROM invoice_line il 
      JOIN track t 
        ON il.track_id = t.track_id
),
usa_track_table AS (
    SELECT i.* 
      FROM invoice_line_track_table i 
      JOIN usa_invoice_table u 
        ON i.invoice_id = u.invoice_id 
), 
metal_genre_table AS ( 
    SELECT genre_id 
      FROM genre 
     WHERE name LIKE '%Metal%' 
), 
metal_usa_track_table AS ( 
    SELECT * 
      FROM usa_track_table u 
      JOIN metal_genre_table m 
        ON u.genre_id = m.genre_id 
) 
       
  SELECT invoice_id AS invoice_id, 
         SUM(quantity * unit_price) AS total, 
         SUM(milliseconds)/1000.0/60 AS minute 
    FROM metal_usa_track_table 
GROUP BY invoice_id


"""

pd.read_sql_query(query,connection)

,invoice_id,total,minute
0,4,1.98,8.788883
1,9,1.98,7.653433
2,17,1.98,9.959167
3,18,0.99,4.379850
4,42,0.99,3.415500
...,...,...,...
61,591,2.97,17.112350
62,595,0.99,4.803467
63,598,1.98,6.268500
64,602,0.99,2.494250


Recursive CTEs Anatomy:

In [10]:
query = """

WITH RECURSIVE
managers_chain(employee_id, path) AS (

-- Create a table of employee_id and name where they don't report to anyone:
SELECT employee_id, 
       last_name || ' ' || first_name AS path 
  FROM employee
 WHERE reports_to IS NULL
 
 UNION ALL
 
SELECT e.employee_id, 
       m.path || '<--' || e.last_name || ' ' || e.first_name AS path
  FROM employee e
  JOIN managers_chain m
    ON e.reports_to = m.employee_id
 )
 
SELECT path
  FROM managers_chain;

"""

pd.read_sql_query(query,connection)

,path
0,Adams Andrew
1,Adams Andrew<--Edwards Nancy
2,Adams Andrew<--Mitchell Michael
3,Adams Andrew<--Edwards Nancy<--Peacock Jane
4,Adams Andrew<--Edwards Nancy<--Park Margaret
5,Adams Andrew<--Edwards Nancy<--Johnson Steve
6,Adams Andrew<--Mitchell Michael<--King Robert
7,Adams Andrew<--Mitchell Michael<--Callahan Laura


## Guided Project: Customers and Products Analysis Using SQL

Please see separate notebook.